# モデル品質モニタリングのステップA

このノートブックを実行する時のヒント：   
- KernelはPython3（Data Science）で動作確認をしています。
- デフォルトではSageMakerのデフォルトBucketを利用します。必要に応じて変更することも可能です。
- 実際に動かさなくても出力を確認できるようにセルのアウトプットを残しています。きれいな状態から実行したい場合は、右クリックメニューから "Clear All Outputs"を選択して出力をクリアしてから始めてください。
- 作成されたスケジュールはSageMaker Studioの`SageMaker resource` （左側ペインの一番下）のEndpointメニューからも確認可能

<span style="color: orange; font-size: 120%; ">モデル名は実行前に設定変更が必要です</span>

In [3]:
# step-0-train-model.ipynb でトレーニングしたモデルの名前に変更してください
model_name = 'nyctaxi-xgboost-regression-2022-12-15-10-58-19-model'

複数のノートブックで共通で使用する変数

In [4]:
# エンドポイント名を指定する
endpoint_name = 'nyctaxi-xgboost-endpoint'

# エンドポイントConfigの名前を指定する
endpoint_config_name = f'{endpoint_name}-config'

# データ品質のモニタリングスケジュールの名前を指定する
model_quality_monitoring_schedule = f'{endpoint_name}-model-quality-schedule'

# SageMaker default bucketをModel Monitorのバケットとして使用
# それ以外のバケットを使用している場合はここで指定する
import sagemaker
bucket = sagemaker.Session().default_bucket()

モニタリング結果を保管するための、ベースラインやレポートのS3上のPrefixを設定します

In [5]:
# ベースラインの出力先Prefixを設定する
baseline_prefix = 'model_monitor/model_quality_baseline'

# 時系列での可視化のために、複数のレポートに共通するPrefixを設定する
report_prefix = 'model_monitor/model_quality_monitoring_report'

# Ground Truthをアップロードする先のPrefixを指定します
ground_truth_prefix = 'model_monitor/model_quality_ground_truth'

## A1. 推論エンドポイントにデータキャプチャの設定を行う

データキャプチャの設定を入れた新しい推論エンドポイントを作成する場合のコードサンプルです。  
<span style="color: orange; ">データ品質ですでにOUTPUT設定付きのエンドポイントを作成している場合はこのセルの実行は不要なので、A2から進めてください</span>

In [6]:
import boto3
import time
import sagemaker

In [ ]:
sm_client = boto3.client('sagemaker')

# Create endpoint config
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialVariantWeight': 1,
        'InitialInstanceCount': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}],
    # Set data capture config
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': f's3://{bucket}/model_monitor/endpoint-data-capture',
        'CaptureOptions': [{'CaptureMode': 'Input'}, {'CaptureMode': 'Output'}],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': ['text/csv'],
            'JsonContentTypes': ['application/json']
        }
    }
)

In [ ]:
def wait_for_endpoint_creation(endpoint_name):
    sm_client = boto3.client('sagemaker')
    
    # Check endpoint creation status
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    while status=='Creating':
        print("Status: " + status)
        time.sleep(60)
        resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
        status = resp['EndpointStatus']
        
    print('Finished!', status)

    
# Create endpoint
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
wait_for_endpoint_creation(endpoint_name)

<span style="color: orange; ">エンドポイントのデプロイに10分程度の時間がかかります</span>

### A2. ベースラインを作成する
<span style="color: orange; ">ベースラインの計算には5分程度の時間がかかります</span>

In [10]:
import boto3
import time
import sagemaker
from datetime import datetime

import pandas as pd
from sagemaker import get_execution_role, session, Session
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat


In [11]:
job_timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
baseline_job_name = "model-quality-baseline-job-{}".format(job_timestamp)

role = get_execution_role()

In [12]:
%%time

model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
)

job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=f's3://{bucket}/model_monitor/model_quality_baseline_input/',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = f's3://{bucket}/{baseline_prefix}/',
    problem_type='Regression',
    inference_attribute= "pred", # データセットに含まれる（テスト時の）推論結果を保持したカラム名
    # probability_attribute= "probability", # データセットに含まれる（テスト時の）確信度を保持したカラム名（Regressionでは不要）
    ground_truth_attribute= "pickup_count" # データセットに含まれる（テスト時の）ラベルを保持したカラム名
)
job.wait(logs=False)


Job Name:  model-quality-baseline-job-2022-12-16-10-27-07
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-370828233696/model_monitor/model_quality_baseline_input/', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-1-370828233696/model_monitor/model_quality_baseline/', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................................................................!CPU times: user 366 ms, sys: 21.7 ms, total: 388 ms
Wall time: 6min 2s


#### 作成されたベースラインを確認する

In [13]:
model_quality_monitor.latest_baselining_job.baseline_statistics().body_dict

{'version': 0.0,
 'dataset': {'item_count': 2688,
  'evaluation_time': '2022-12-16T10:32:50.057Z'},
 'regression_metrics': {'mae': {'value': 38.29501488095238,
   'standard_deviation': 0.21881824869307076},
  'mse': {'value': 2809.991443452381, 'standard_deviation': 21.11839953830601},
  'rmse': {'value': 53.00935241495015,
   'standard_deviation': 0.19907891114454224},
  'r2': {'value': 0.9499197226669486,
   'standard_deviation': 0.0006458841438745311}}}

In [14]:
model_quality_monitor.latest_baselining_job.suggested_constraints().body_dict

{'version': 0.0,
 'regression_constraints': {'mae': {'threshold': 38.29501488095238,
   'comparison_operator': 'GreaterThanThreshold'},
  'mse': {'threshold': 2809.991443452381,
   'comparison_operator': 'GreaterThanThreshold'},
  'rmse': {'threshold': 53.00935241495015,
   'comparison_operator': 'GreaterThanThreshold'},
  'r2': {'threshold': 0.9499197226669486,
   'comparison_operator': 'LessThanThreshold'}}}

## A3. スケジュールの作成

In [15]:
import sagemaker
from sagemaker import model_monitor
from sagemaker.model_monitor import Constraints, EndpointInput

bucket = sagemaker.Session().default_bucket()
model_constraints = Constraints.from_s3_uri(f's3://{bucket}/{baseline_prefix}/constraints.json')

In [16]:
model_quality_monitor = ModelQualityMonitor(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
)

# ここでは、推論からGround Truthの収集までに1時間から3時間かかることを想定したオフセットをセットする
# テスト目的ですぐにGround Truthをセットしてアップロードする場合は、end_time_offest="-PT0H"を指定する
endpont_input = EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",
        start_time_offset="-PT3H",
        end_time_offset="-PT0H",
        inference_attribute="0",
)

model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_quality_monitoring_schedule,
    output_s3_uri=f's3://{bucket}/{report_prefix}',
    constraints=model_constraints,
    schedule_cron_expression=model_monitor.CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
    endpoint_input=endpont_input,
    ground_truth_input=f's3://{bucket}/{ground_truth_prefix}',
    problem_type='Regression',
)

作成されたスケジュールを確認

In [17]:
sm_client = boto3.client('sagemaker')
sm_client.describe_monitoring_schedule(MonitoringScheduleName=model_quality_monitoring_schedule)

{'MonitoringScheduleArn': 'arn:aws:sagemaker:ap-northeast-1:370828233696:monitoring-schedule/nyctaxi-xgboost-endpoint-model-quality-schedule',
 'MonitoringScheduleName': 'nyctaxi-xgboost-endpoint-model-quality-schedule',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2022, 12, 16, 10, 33, 11, 742000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 12, 16, 10, 33, 11, 849000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2022-12-16-10-33-11-430',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'nyctaxi-xgboost-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'nyctaxi-xgboost-endpoint-model-quality-schedule',
  'ScheduledTime': datetime.datetime(2022, 12, 16, 9, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2022, 12, 16, 9, 8, 25, 258000, tzi